In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import math
import h5py, glob
import pandas as pd

from scipy.stats import ks_2samp
from ang_res_funcs import *

In [ ]:
data = {'year': [2012_original, 2012_original, 2012, 2012, 2015, 2015, 2018, 2018], 
        'tier': [3, 4, 3, 4, 3, 4, 3, 4],
        'energy': [100, 200, 300, 150, 250, 350, 200, 300, 400]}
df = pd.DataFrame(data)